In [13]:
import pandas as pd

In [14]:
DATA_FOLDER = "./data/"
# DENSITY03 = "02_CDensity_1_201903.csv.gz"
# DENSITY04 = "02_CDensity_1_201904.csv.gz"

In [15]:
final_csv = pd.read_csv(DATA_FOLDER + "final_df.csv")

In [16]:
final_csv['date'] = pd.to_datetime(final_csv.date)

In [17]:
final_csv['date_dt'] = final_csv.date.dt.date

In [18]:
final_csv_filtered = final_csv.loc[(final_csv.date_dt >= pd.to_datetime("2020-03-01")) & (final_csv.date_dt < pd.to_datetime("2020-05-01"))]

In [19]:
final_csv_filtered

,date,address,Point,date_dt
36,2020-04-21 14:42:11.009000+00:00,"Город Москва, Проезд Тушинский 1-й Дом 6 Корпус 1",37.421515 55.833693,2020-04-21
39,2020-04-22 16:36:34.901000+00:00,"Область Московская, Город Озеры, Улица Ленина ...",38.564612 54.852537,2020-04-22
82,2020-04-27 04:21:21.068000+00:00,"Город Москва, Улица 800-летия Москвы Дом 4 Кор...",37.534909 55.875551,2020-04-27
83,2020-04-27 04:11:14.797000+00:00,"Город Москва, Улица 800-летия Москвы Дом 4 Кор...",37.534909 55.875551,2020-04-27
112,2020-04-24 18:19:23.469000+00:00,"Город Москва, Улица Митинская",37.36387 55.84537,2020-04-24
...,...,...,...,...
227604,2020-04-24 06:48:35.929000+00:00,"город москва, большой афанасьевский переулок, ...",NaN,2020-04-24
227613,2020-04-23 18:58:46.693000+00:00,"город москва, улица новая башиловка, дом 8",NaN,2020-04-23
227616,2020-04-27 11:11:20.876000+00:00,"город москва, большая почтовая улица, дом 18/2...",37.69136 55.773692,2020-04-27
227617,2020-04-25 14:19:46.413000+00:00,"город москва, большая почтовая улица, дом 18/2...",37.69136 55.773692,2020-04-25


In [20]:
address_cell_id = pd.read_csv(DATA_FOLDER + "address_cell_id.csv")

In [21]:
result_df = final_csv_filtered.merge(address_cell_id[["address", "cell_id"]], on=["address"], how="left")

In [22]:
result_df.isnull().sum()

date          0
address       0
Point      3098
date_dt       0
cell_id    3098
dtype: int64

In [23]:
res = result_df.dropna(subset=["Point"], axis=0)

In [24]:
trg_df = res.groupby(["date_dt", "cell_id"])["date"].count().reset_index()

In [25]:
trg_df = trg_df.rename({"date":"target", "cell_id":"zid"}, axis=1)

In [26]:
trg_df['day'] = trg_df.date_dt.transform(lambda x: x.day)

In [27]:
agg_density04 = pd.read_parquet(DATA_FOLDER + "aggregated_density04.parquet")

In [28]:
agg_density04

,day,zid,customers_cnt_total,customers_cnt_long,customers_cnt_work,customers_cnt_loc,month
0,1,1,7.916667,7.916667,0.000000,7.916667,4
1,1,2,10.354167,9.312500,0.000000,9.229167,4
2,1,3,15.937500,15.645833,0.000000,15.791667,4
3,1,4,842.395833,650.708333,103.812500,443.687500,4
4,1,5,4714.250000,4105.791667,576.062500,3738.208333,4
...,...,...,...,...,...,...,...
5298362,30,190238,2.850000,2.100000,0.000000,0.000000,4
5298363,30,190239,0.000000,0.000000,0.000000,0.000000,4
5298364,30,190240,118.104167,99.583333,4.520833,105.416667,4
5298365,30,190241,2.000000,1.958333,0.000000,2.000000,4


In [20]:
train_df = agg_density04.merge(trg_df[["day", "zid", "target"]], on=["day", "zid"], how="inner")

In [21]:
train_df

,day,zid,customers_cnt_total,customers_cnt_long,customers_cnt_work,customers_cnt_loc,month,target
0,4,4176,3776.354167,3097.958333,984.104167,1819.666667,4,1
1,4,8814,2596.645833,2117.208333,391.479167,1808.729167,4,1
2,4,21765,4634.666667,4083.354167,541.250000,3745.520833,4,1
3,4,37096,4526.750000,3748.625000,877.312500,3092.604167,4,1
4,4,43184,4035.375000,3357.541667,484.312500,2857.208333,4,1
...,...,...,...,...,...,...,...,...
21776,30,189604,4964.250000,3929.166667,782.479167,2980.000000,4,1
21777,30,189645,2720.937500,2394.958333,234.979167,2203.104167,4,1
21778,30,189974,2651.062500,2197.833333,340.479167,2146.125000,4,3
21779,30,190081,3016.354167,2577.500000,311.229167,2395.750000,4,1


In [22]:
import geopandas as gp

In [23]:
neighbors = pd.read_parquet(DATA_FOLDER + "fisnet_with_neighbors.parquet")

In [24]:
train_df = train_df.merge(neighbors[["cell_id", "neighbors_list"]], left_on='zid', right_on='cell_id', how='left').drop("cell_id", axis=1)

In [25]:
train_df

,day,zid,customers_cnt_total,customers_cnt_long,customers_cnt_work,customers_cnt_loc,month,target,neighbors_list
0,4,4176,3776.354167,3097.958333,984.104167,1819.666667,4,1,"[122071, 61371, 174319, 178073, 63606, 49204, ..."
1,4,8814,2596.645833,2117.208333,391.479167,1808.729167,4,1,"[76984, 177459, 44588, 162228, 138843, 142788,..."
2,4,21765,4634.666667,4083.354167,541.250000,3745.520833,4,1,"[185718, 1535, 14551, 166886, 152565, 132135, ..."
3,4,37096,4526.750000,3748.625000,877.312500,3092.604167,4,1,"[149871, 4924, 92127, 143668, 111069, 31677, 6..."
4,4,43184,4035.375000,3357.541667,484.312500,2857.208333,4,1,"[140346, 38681, 97522, 44971, 92672, 8328, 648..."
...,...,...,...,...,...,...,...,...,...
21776,30,189604,4964.250000,3929.166667,782.479167,2980.000000,4,1,"[180175, 3450, 184227, 21825, 42074, 149187, 1..."
21777,30,189645,2720.937500,2394.958333,234.979167,2203.104167,4,1,"[6851, 12379, 126550, 112373, 68262, 126101, 8..."
21778,30,189974,2651.062500,2197.833333,340.479167,2146.125000,4,3,"[92377, 9970, 160396, 52584, 174906, 11987, 17..."
21779,30,190081,3016.354167,2577.500000,311.229167,2395.750000,4,1,"[122937, 159545, 123160, 57990, 38561, 16979, ..."


In [26]:
train_df.to_parquet(DATA_FOLDER + "train_df.parquet")